In [1]:
# | default_exp core.handoffs

In [2]:
# | export
from typing import Dict, Any, Optional, List, Callable
from dataclasses import dataclass
from enum import Enum
import json


class HandoffType(Enum):
    TRANSFER = "transfer"
    DELEGATE = "delegate" 
    COLLABORATE = "collaborate"


@dataclass
class HandoffRequest:
    """Request to handoff to another agent"""
    target_agent: str
    handoff_type: HandoffType
    context: Dict[str, Any]
    message: str
    return_to_sender: bool = True


@dataclass
class HandoffResponse:
    """Response from a handoff"""
    success: bool
    result: Dict[str, Any]
    agent_name: str
    error: Optional[str] = None


class HandoffManager:
    """Manages handoffs between agents"""
    
    def __init__(self):
        self.agents: Dict[str, Any] = {}  # Agent registry
        self.handoff_history: List[Dict] = []
        self.active_handoffs: Dict[str, HandoffRequest] = {}
    
    def register_agent(self, name: str, agent):
        """Register an agent for handoffs"""
        self.agents[name] = agent
    
    def execute_handoff(self, request: HandoffRequest) -> HandoffResponse:
        """Execute a handoff request"""
        if request.target_agent not in self.agents:
            return HandoffResponse(
                success=False,
                result={},
                agent_name=request.target_agent,
                error=f"Agent '{request.target_agent}' not found"
            )
        
        target_agent = self.agents[request.target_agent]
        
        try:
            # Prepare context for target agent
            handoff_message = self._prepare_handoff_message(request)
            
            # Execute on target agent
            result = target_agent.run(handoff_message)
            
            # Record handoff
            self._record_handoff(request, result)
            
            return HandoffResponse(
                success=True,
                result=result,
                agent_name=request.target_agent
            )
            
        except Exception as e:
            return HandoffResponse(
                success=False,
                result={},
                agent_name=request.target_agent,
                error=str(e)
            )
    
    def _prepare_handoff_message(self, request: HandoffRequest) -> str:
        """Prepare message for handoff"""
        context_str = json.dumps(request.context, indent=2) if request.context else "No additional context"
        
        handoff_message = f"""
HANDOFF REQUEST ({request.handoff_type.value.upper()})

Original Message: {request.message}

Context:
{context_str}

Please process this request and provide a response.
"""
        return handoff_message
    
    def _record_handoff(self, request: HandoffRequest, result: Dict[str, Any]):
        """Record handoff in history"""
        self.handoff_history.append({
            "target_agent": request.target_agent,
            "handoff_type": request.handoff_type.value,
            "message": request.message,
            "context": request.context,
            "result": result,
            "timestamp": self._get_timestamp()
        })
    
    def _get_timestamp(self) -> str:
        """Get current timestamp"""
        from datetime import datetime
        return datetime.now().isoformat()


class HandoffTool:
    """Tool for agents to initiate handoffs"""
    
    def __init__(self, handoff_manager: HandoffManager, current_agent_name: str):
        self.handoff_manager = handoff_manager
        self.current_agent_name = current_agent_name
    
    @property
    def name(self) -> str:
        return "handoff"
    
    @property
    def description(self) -> str:
        return "Transfer conversation to another specialized agent"
    
    @property
    def parameters(self) -> Dict[str, Any]:
        return {
            "type": "object",
            "properties": {
                "target_agent": {
                    "type": "string",
                    "description": "Name of the agent to handoff to"
                },
                "handoff_type": {
                    "type": "string",
                    "enum": ["transfer", "delegate", "collaborate"],
                    "description": "Type of handoff"
                },
                "message": {
                    "type": "string",
                    "description": "Message to send to the target agent"
                },
                "context": {
                    "type": "object",
                    "description": "Additional context for the handoff"
                }
            },
            "required": ["target_agent", "message"]
        }
    
    def execute(self, target_agent: str, message: str, 
                handoff_type: str = "transfer", context: Optional[Dict] = None) -> Dict[str, Any]:
        """Execute handoff"""
        request = HandoffRequest(
            target_agent=target_agent,
            handoff_type=HandoffType(handoff_type),
            message=message,
            context=context or {}
        )
        
        response = self.handoff_manager.execute_handoff(request)
        
        return {
            "success": response.success,
            "target_agent": response.agent_name,
            "result": response.result,
            "error": response.error
        }

